# Guess Who…
<small>…has Oracle access to HRDW1 tables?</small>

## Real-World Scenario: HRDW1

- Which users have access to HRDW1?

- Which RDA accounts have access to HRDW1?

- Who has access to the `hrdw1.job` table?

<h2>
  <span style="color: #ccc; font-size: .75em;">First Idea</span><br>
  Access Validation
</h2>

`acdw.user_data` & `acdw.user_role`

| User | Role | System |
| :--- | :--- | :---: |
| JAMJAM | 9F0893E0-ACDA-4113-B3D7-DE745F0D1BE3 | MRPT |
| JAMJAM | A User WebIntelligence XI | DW |
| JAMJAM | DAA University Housing Modify | DW |
| JAMJAM | IQ\_PROD\_SUPPORT\_SEL | DW |
| JAMJAM | M\_RADW1\_SEL | DW |
| JAMJAM | M\_SRDW1\_ADDR\_SEL | DW |
| JAMJAM | Stu Admiss\_Def | DW |
| JAMJAM | Stu Life Exp\_Def | DW |

**A couple of issues**

- Doesn't say which tables belong to a role

- Only shows top-level roles assigned to people

- Does not contain info about RDA access

<h2>
  <span style="color: #ccc; font-size: .75em;">Second Idea</span><br>
  Query Oracle Security Tables
</h2>

## How permissions work in Oracle

![img](static/grants.svg)

## How to query Oracle permissions

### Three tables to know

#### `all_users`
All user accounts in the database

#### `dba_role_privs`
Mapping of `grantee` to `granted_role` where `grantee` can be either a user or a role

#### `dba_tab_privs`
Mapping of `grantee` to a specific table (`owner`.`table_name`) and `privelege` (`select`, `delete`, etc.)

### One Gotcha... Recursive Joins

![recursive joins](static/recursive-links.svg)

Recursive joins via `start with` & `connect by`

```sql
with hrdw1_folks as (
    select grantee from dba_tab_privs where owner = 'M_HRDW1'
)

select
  grantee,
  granted_role
from dba_role_privs
  start with granted_role in (select grantee from hrdw1_folks)
    connect by prior grantee = granted_role
```

References:
- [Oracle's Pretty Docs](https://livesql.oracle.com/apex/livesql/file/tutorial_GQMLEEPG5ARVSIFGQRD3SES92.html)
- [Oracle's Ugly Docs](https://docs.oracle.com/en/database/oracle/oracle-database/21/sqlrf/Hierarchical-Queries.html#GUID-0118DF1D-B9A9-41EB-8556-C6E7D6A5A84E)

How we can (prettily) identify all users who have access to a particular schema:

<div style="--jp-code-font-size: 10px;">
    
```sql
with table_grants as (
  -- collection of roles that have been granted schema access
  select distinct
    grantee,
    owner
  from dba_tab_privs
  where
    owner = :schema
),

permissions as (
  -- build permissions tree through recursion
  select
    grantee,
    granted_role,
    connect_by_root granted_role as root,
    substr(sys_connect_by_path(granted_role, '.'), 2) as path
  from dba_role_privs
    start with granted_role in (select grantee from table_grants)
      connect by granted_role = prior grantee

  union

  -- create dummy permission for direct grants
  select
    username as grantee,
    username as granted_role,
    username as root,
    username as path
  from all_users
  where
    username in (select grantee from table_grants)
),

final as (
  -- tidy up output
  select
    :schema as schema,
    a.grantee as username,
    listagg(distinct a.path, chr(10)) within group (order by a.path) as permissions,
    listagg(distinct b.table_name, chr(10)) within group (order by b.table_name) as tables
  from permissions a
    join dba_tab_privs b on a.root = b.grantee and b.owner = :schema
  group by a.grantee
)

select * from final order by username
```
    
</div>

| Schema | Username | Permissions | Tables |
| :--- | :--- | :--- | :--- |
| M_HRDW1 | JAMJAM | ACDW1_DEF_SEL.IQ_PROD_SUPPORT_SEL<br>IQ_PROD_SUPPORT_SEL<br>M_CRDW1_DEF_SEL.IQ_PROD_SUPPORT_SEL<br>M_HRDW1_MARKET_SEL.IQ_PROD_SUPPORT_SEL<br>M_PYDW1_EMPL_CONTRIB_SEL.IQ_PROD_SUPPORT_SEL<br>M_TLDW1_DEF_SEL.IQ_PROD_SUPPORT_SEL |  ADDL_PAY_DATA<br>ADMINISTRATIVE_PMODS<br>CONTRACT<br>DBE_MONTHLY_CROSSTAB_VW<br>DEPT_BUDGET_ERN<br>... |
| M_HRDW1 |RMTBUSSCHOOL_DWPROD5 | RMTBUSSCHOOL_DWPROD5_SEL | HE_DEPT_TBL |
| … | … | … | … |
